In [ ]:
import os
import pandas as pd
import numpy as np
from torchvision.io import read_image
from torchvision.models import squeezenet1_1, SqueezeNet1_1_Weights
from functools import wraps
import time

In [ ]:
def timeit(func):
    @wraps(func)
    def timeit_wrapper(*args, **kwargs):
        start_time = time.perf_counter()
        result = func(*args, **kwargs)
        end_time = time.perf_counter()
        total_time = end_time - start_time
        print(f'Function {func.__name__}{args} {kwargs} Took {total_time:.4f} seconds')
        return result
    return timeit_wrapper

In [ ]:
@timeit
def squeezenet_detector(path):
  # Step 1: Initialize model with the best available weights
  weights = SqueezeNet1_1_Weights.IMAGENET1K_V1
  model = squeezenet1_1(weights=weights)
  model.eval()

  # Step 2: Initialize the inference transforms
  preprocess = weights.transforms()

  # Creates a list of directories for each image
  directories = []
  for path, subdirs, files in os.walk(path):
    for name in files:
      directories.append(os.path.join(path, name))

  # Creates an empty dataframe
  df = pd.DataFrame()

  for file_name in directories:
    try:
      # Step 3: Apply inference preprocessing transforms
      batch = preprocess(read_image(file_name)).unsqueeze(0)
      # Step 4: Use the model and print the predicted category
      prediction = model(batch).squeeze(0).softmax(0)
      class_id = prediction.argmax().item()
      score = prediction[class_id].item()
      category_name = weights.meta["categories"][class_id]

      # append each result in the dataframe
      row = {
          'img_path': [file_name],
          'category': [category_name],
          'cat_score': [round(100 * score, 2)],
          }

      df = pd.concat([df, pd.DataFrame(row)])
    except Exception as e:
      print(f'Exception {e} with {file_name}')
  return df

In [ ]:
df = squeezenet_detector('/content/data')
df

Function squeezenet_detector('/content/data',) {} Took 12.5232 seconds


,img_path,category,cat_score
0,/content/data/carpeta2/896_2_37cb1c29-1d5b-484...,convertible,28.28
0,/content/data/carpeta2/902_8_e093518e-9c17-419...,fountain pen,23.22
0,/content/data/carpeta2/901_12_3ac2c75c-121a-05...,seat belt,7.85
0,/content/data/carpeta2/892_12_d68b8307-2005-41...,seat belt,22.27
0,/content/data/carpeta2/899_13_d0d8547d-5b4a-47...,Polaroid camera,14.18
...,...,...,...
0,/content/data/carpeta1/905_14_7d71c74b-ed3d-c6...,paper towel,21.52
0,/content/data/carpeta1/902_10_e093518e-9c17-41...,car wheel,71.20
0,/content/data/carpeta1/889_11_a278a24b-3424-cf...,sports car,68.17
0,/content/data/carpeta1/896_13_37cb1c29-1d5b-48...,dial telephone,24.35


In [ ]:
from torchvision.io import read_file